
# Notebook de Validación — `ClassifierAgent`

Este cuaderno valida el comportamiento del agente clasificador (`classifier_agent.py`) usando un conjunto de **prompts simulados**.



## 1) Requisitos previos

- Tener instalado Python 3.10+
- Instalar dependencias mínimas:

```bash
pip install langchain openai tiktoken
```

> Si vas a usar **OpenRouter**, exporta tu clave de API en el entorno:
```bash
export OPENROUTER_API_KEY="tu_api_key_aqui"
```


In [1]:
!pip install -r requirements.txt -q

In [10]:
%pip install -U langchain langchain-openai langchain-community tiktoken pandas

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.74-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.4.15-py3-none-any.whl.metadata (14 kB)
  Using cached openai-1.100.2-py3-none-any.whl.metadata (29 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
   ---------------------------------------- 0.0/74.4 kB ? eta -:--:--
   ---------------------------------------- 74.4/74.4 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.5 MB 9.6 MB/s eta 0:00:01
   ------------------ --------------------- 1.2/2.5 MB 10.6 MB/s eta 0:00:01
   ---------

  You can safely remove it manually.
  You can safely remove it manually.


In [9]:
import os
print(os.getenv("OPENROUTER_API_KEY"))

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


In [10]:

import os
import pandas as pd
from pathlib import Path

# Mostrar versiones útiles
import sys
print("Python:", sys.version)

# Comprobar que existen los archivos necesarios
base = Path('.')
csv_path = Path('data/prompts_simulados.csv')
agent_path = Path('agents/classifier_agent.py')

print("Existe prompts_simulados.csv:", csv_path.exists())
print("Existe classifier_agent.py:", agent_path.exists())


Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
Existe prompts_simulados.csv: True
Existe classifier_agent.py: True


In [11]:
import importlib.util, sys
from pathlib import Path

agent_path = Path("agents/classifier_agent.py")  # ajusta la ruta si está en /agents
spec = importlib.util.spec_from_file_location("classifier_agent", str(agent_path))
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)
ClassifierAgent = mod.ClassifierAgent

agent = ClassifierAgent()
agent.classify("Me voy a mudar porque no soporto esta ciudad")


'4. Cambio de comunidad por motivos generales'


## 2) Instanciar el agente

Por defecto usaremos un modelo compatible con OpenRouter. Puedes ajustar el `model_name` si lo deseas.


In [12]:

# Comprueba que tu variable OPENROUTER_API_KEY está configurada
if not os.getenv("OPENROUTER_API_KEY"):
    print("⚠️  No se encontró la variable de entorno OPENROUTER_API_KEY. "
          "Configúrala si vas a usar OpenRouter.")
    
agent = ClassifierAgent(model_name="gpt-3.5-turbo", temperature=0.1)
agent



## 3) Cargar dataset de prompts simulados


In [13]:

df = pd.read_csv(csv_path)
df.head()


,prompt,categoria_esperada
0,"Me mudo de Sevilla a Zaragoza con mis hijos, ¿...",Mudanza con hijos
1,"Voy a cambiar mi residencia por trabajo, sin h...",Cambio de comunidad por trabajo
2,"He decidido mudarme a otra comunidad, pero no ...",Mudanza sin hijos
3,"Después de jubilarme, me traslado al norte. ¿Q...",Otro
4,Nos mudamos toda la familia a Galicia. Necesit...,Mudanza con hijos



## 4) Ejecutar clasificación y evaluar resultados

- Se ejecuta `agent.classify()` sobre cada prompt.
- Se compara la categoría devuelta por el modelo contra `categoria_esperada`.
- Se calcula una métrica simple de **accuracy** y una **tabla de confusión**.


In [14]:

import re

def normalize(label: str) -> str:
    if not isinstance(label, str):
        return "otro"
    s = label.strip().lower()
    # Normalizaciones básicas por si el modelo devuelve variaciones
    if "hijo" in s:
        return "mudanza con hijos"
    if ("trabajo" in s) or ("labor" in s):
        return "cambio de comunidad por trabajo"
    if ("sin hijo" in s) or ("solo" in s) or ("personal" in s):
        return "mudanza sin hijos"
    if s in {"1", "mudanza con hijos"}:
        return "mudanza con hijos"
    if s in {"2", "mudanza sin hijos"}:
        return "mudanza sin hijos"
    if s in {"3", "cambio de comunidad por trabajo"}:
        return "cambio de comunidad por trabajo"
    return "otro"

preds = []
for i, row in df.iterrows():
    text = row["prompt"]
    try:
        y = agent.classify(text)
    except Exception as e:
        print(f"Error clasificando fila {i}: {e}")
        y = "error"
    preds.append(y)

df["pred_raw"] = preds
df["pred"] = df["pred_raw"].apply(normalize)
df["esperada_norm"] = df["categoria_esperada"].apply(lambda x: x.strip().lower())

acc = (df["pred"] == df["esperada_norm"]).mean()
print(f"Accuracy simple: {acc:.2%}")
df[["prompt","categoria_esperada","pred_raw","pred"]]


Accuracy simple: 75.00%


,prompt,categoria_esperada,pred_raw,pred
0,"Me mudo de Sevilla a Zaragoza con mis hijos, ¿...",Mudanza con hijos,1. Mudanza con hijos,mudanza con hijos
1,"Voy a cambiar mi residencia por trabajo, sin h...",Cambio de comunidad por trabajo,Cambio de comunidad por trabajo,cambio de comunidad por trabajo
2,"He decidido mudarme a otra comunidad, pero no ...",Mudanza sin hijos,Mudanza sin hijos,mudanza con hijos
3,"Después de jubilarme, me traslado al norte. ¿Q...",Otro,Cambio de comunidad por motivos generales,otro
4,Nos mudamos toda la familia a Galicia. Necesit...,Mudanza con hijos,1. Mudanza con hijos,mudanza con hijos
5,Cambio de comunidad por una nueva oferta laboral.,Cambio de comunidad por trabajo,Cambio de comunidad por trabajo,cambio de comunidad por trabajo
6,Me traslado solo por motivos personales.,Mudanza sin hijos,Cambio de comunidad por motivos generales,otro
7,Mudanza con hijos en edad escolar. Necesito sa...,Mudanza con hijos,1. Mudanza con hijos,mudanza con hijos


In [15]:

cm = pd.crosstab(df["esperada_norm"], df["pred"], rownames=["Esperada"], colnames=["Predicha"], dropna=False)
cm


Predicha,cambio de comunidad por trabajo,mudanza con hijos,otro
Esperada,,,
cambio de comunidad por trabajo,2,0,0
mudanza con hijos,0,3,0
mudanza sin hijos,0,1,1
otro,0,0,1



## 5) Guardar resultados

Se exporta un CSV con las columnas clave para auditoría.


In [16]:

out_path = Path("resultados_clasificacion.csv")
df.to_csv(out_path, index=False)
print("Resultados guardados en:", out_path.resolve())


Resultados guardados en: C:\Users\loloa\Documents\UNIR_LAB\TFM\produccion\TFM\resultados_clasificacion.csv
